In [1]:
from run_early_visual_models_principal_distortions import get_default_args
import run_early_visual_models_principal_distortions
import argparse 
from plotting_functions import * 
from run_early_visual_models_principal_distortions import load_pckl_file as load_pckl_file_early_visual_models

%load_ext autoreload
%autoreload 2

/mnt/sw/nix/store/gpkc8q6zjnp3n3h3w9hbmbj6gjbxs85w-python-3.10.10-view/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/sw/nix/store/6qvrglgqdpwhbw9zv2nh07fpd7a4wq31-py-torchvision-0.15.2/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Timm models not installed. Please pip install timm if you want to use these models


In [2]:
test_models = ['LN',
               'LG',
               'LGG',
               'LGN',
                ]

model_names = ';'.join(test_models)

print(model_names)

LN;LG;LGG;LGN


In [5]:
short_model_name = 'early_visual_models'

all_args = get_default_args()
custom_args = {
    'model_names': model_names, 
    'image_path': 'images/parrot.png', ### YOUR IMAGE PATH HERE, NOTE: THESE MODELS OPERATE ON BLACK AND WHITE IMAGES ###,
    'num_steps':5000,
    'output_custom_exp_name':short_model_name,    
    'lr_start_end':[-1,-3],
    'seed':0,
    
    'perturb_clamp_scale':0.01,
}

all_args.update(custom_args)
all_args = argparse.Namespace(**all_args)

In [ ]:
# Run the optimization
run_early_visual_models_principal_distortions.main(all_args)

### Additional plots, such as the optimization and simulating the model scaling

In [6]:
output = load_pckl_file_early_visual_models(all_args)

plt.figure(figsize=(2,2))
plt.plot([d.detach().cpu().numpy() for d in output['objective_hist']])
plt.title('Optimization Loss')

FileNotFoundError: Pickle file not found at ./results/early_visual_models/parrot_lr_-1_-3_steps_5000_norm_0.1_clamp_0.01_gaussian_False_cutoff_1000_seed_0

In [7]:
from models.early_vision_models import *
device = 'cuda'

model_list = []
for model_name in test_models:
    model = EARLY_VISUAL_MODEL_DICT[model_name](device=device)
    model.to(device)
    model_list.append(model)

In [8]:
test_images = [all_args.image_path,
              ]

show_examples_with_varying_perturbations(all_args, 
                                         test_images, 
                                         model_list=model_list,
                                         vertical_layout=False,
                                         perturb_scales=[500,250],
                                         model_for_positive_thresh=model_list[0],
                                         )

FileNotFoundError: Pickle file not found at ./results/early_visual_models/parrot_lr_-1_-3_steps_5000_norm_0.1_clamp_0.01_gaussian_False_cutoff_1000_seed_0

<Figure size 4800x600 with 0 Axes>

In [ ]:
base_distortion = 500

test_image = output['test_image']
opt_perturbation_a = output['opt_perturbation_a']
opt_perturbation_b = output['opt_perturbation_b']

model_order = [
               'LuminanceContrastGainControl (LGG)',
               'LinearNonLinear (LN)',
               'OnOff (LGN)',
               'LuminanceGainControl (LG)',
               ]


plt.figure(figsize=(12,4*len(model_list)))

for model in model_list:
    m_idx = model_order.index(model.model_name)
    t_1, t_2 = utils.compute_model_thresh(model, test_image, opt_perturbation_a, opt_perturbation_b)
    ratio = ch.sqrt(t_1) / ch.sqrt(t_2)
    factor_a = base_distortion * ratio
    factor_b = base_distortion / ratio
        
    model_normalization = 0.5 * (factor_a + factor_b)
    factor_a /= (model_normalization)
    factor_b /= (model_normalization)
        
    total_mul_fac_a = base_distortion*factor_a
    total_mul_fac_b = base_distortion*factor_b
    
    dist_a, dist_b = utils.compute_l2_distance(model,
                                           test_image,
                                           total_mul_fac_a*opt_perturbation_a,
                                           total_mul_fac_b*opt_perturbation_b)
    
    ax = plt.subplot(len(model_list),3,1+3*m_idx)
    ax.text(0.6,.5,f'{model.model_name}',
              bbox={'facecolor':'white','alpha':1,'edgecolor':'none','pad':-10},
              ha='center', va='center', fontsize=12) 
    plt.axis('off')
    
    ax = plt.subplot(len(model_list),3,2+3*m_idx)
    perturbed_image_a = np.rollaxis((test_image + base_distortion*opt_perturbation_a*factor_a).detach().cpu().numpy()[0,:,:,:], 0, 3)
    plt.imshow(perturbed_image_a,
               cmap='gray', interpolation='none', vmin=0, vmax=1)
    
    # Only label the leftmost distortion
    scaled_total_mul_fac_a = total_mul_fac_a * ch.norm(opt_perturbation_a)
    plt.title(f'{scaled_total_mul_fac_a:0.3f} * \epsilon 1')
    plt.axis('off')
    
    ax = plt.subplot(len(model_list),3,3+3*m_idx)
    perturbed_image_b = np.rollaxis((test_image + base_distortion*opt_perturbation_b*factor_b).detach().cpu().numpy()[0,:,:,:], 0, 3)
    plt.imshow(perturbed_image_b,
               cmap='gray', interpolation='none', vmin=0, vmax=1)
    plt.axis('off')
    scaled_total_mul_fac_b = total_mul_fac_b * ch.norm(opt_perturbation_b)
    plt.title(f'{scaled_total_mul_fac_b:0.3f} * \epsilon 2')    